In [1]:
import pandas as pd
import os
import requests
import admin
import geocoder as gc
import time
import numpy as np

# Import required libraries for RDD
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats


In [2]:
df1 = admin.read_csv_files(r"C:\Users\THW_9\Desktop\LSE School\Dis\REALIS DATA\2324")

In [3]:
df1 ['Street_Address'] = df1['Address'].str.split(' #').str[0]
df_street_address = df1.groupby('Street_Address').size().reset_index(name='count')

In [4]:
df_street_address

,Street_Address,count
0,1 AKYAB ROAD,1
1,1 ALEXANDRA VIEW,16
2,1 AMBER GARDENS,10
3,1 AMBER ROAD,4
4,1 ANG MO KIO STREET 66,3
...,...,...
8600,9E HILLCREST ROAD,1
8601,9G YUAN CHING ROAD,2
8602,9L SURIN AVENUE,1
8603,9Q HILLCREST ROAD,1


In [5]:
df_dist = pd.read_csv(r"C:\Users\THW_9\Desktop\LSE School\Dis\GIS\Historic\street_addresses_1846RD[increased2324].csv")

In [6]:
# replace NaN with zeros in 'Polygon' column
df_dist['Polygon'] = df_dist['Polygon'].fillna(0)

df_dist['signed_distance'] = df_dist['Distance'] * (df_dist['Polygon'] - 1)  # Create a new column for signed distance

#if polygon is 1, make distance negative
df_dist.loc[df_dist['Polygon'] == 1, 'signed_distance'] = df_dist['Distance']

In [7]:
#clean up for duplicate 'InputID'

df_dist = df_dist.drop_duplicates(subset=['InputID'])
df_dist

,InputID,TargetID,Distance,Polygon,signed_distance
0,1 AKYAB ROAD,1,1399.349343,0.0,-1399.349343
2,1 ALEXANDRA VIEW,1,555.745422,0.0,-555.745422
4,1 AMBER GARDENS,1,3569.721969,0.0,-3569.721969
6,1 AMBER ROAD,1,3647.607627,0.0,-3647.607627
8,1 ANG MO KIO STREET 66,1,8304.079230,0.0,-8304.079230
...,...,...,...,...,...
17144,9E HILLCREST ROAD,1,1893.383759,0.0,-1893.383759
17146,9G YUAN CHING ROAD,1,10791.891099,0.0,-10791.891099
17148,9L SURIN AVENUE,1,5518.345053,0.0,-5518.345053
17150,9Q HILLCREST ROAD,1,1951.778451,0.0,-1951.778451


In [8]:
# merge df1 'Address_Rd' with df_dist 'InputID'
df2 = df_street_address.merge(df_dist, left_on='Street_Address', right_on='InputID', how='left')

In [9]:
df2

,Street_Address,count,InputID,TargetID,Distance,Polygon,signed_distance
0,1 AKYAB ROAD,1,1 AKYAB ROAD,1.0,1399.349343,0.0,-1399.349343
1,1 ALEXANDRA VIEW,16,1 ALEXANDRA VIEW,1.0,555.745422,0.0,-555.745422
2,1 AMBER GARDENS,10,1 AMBER GARDENS,1.0,3569.721969,0.0,-3569.721969
3,1 AMBER ROAD,4,1 AMBER ROAD,1.0,3647.607627,0.0,-3647.607627
4,1 ANG MO KIO STREET 66,3,1 ANG MO KIO STREET 66,1.0,8304.079230,0.0,-8304.079230
...,...,...,...,...,...,...,...
8600,9E HILLCREST ROAD,1,9E HILLCREST ROAD,1.0,1893.383759,0.0,-1893.383759
8601,9G YUAN CHING ROAD,2,9G YUAN CHING ROAD,1.0,10791.891099,0.0,-10791.891099
8602,9L SURIN AVENUE,1,9L SURIN AVENUE,1.0,5518.345053,0.0,-5518.345053
8603,9Q HILLCREST ROAD,1,9Q HILLCREST ROAD,1.0,1951.778451,0.0,-1951.778451


In [10]:
df_MRTdist = pd.read_csv(r"C:\Users\THW_9\Desktop\LSE School\Dis\GIS\Historic\street_addresses_1846RD_reproject_k2MRTdist[2324].csv")

In [11]:
df_MRTdist

,InputID,TargetID,Distance
0,1 AKYAB ROAD,NOVENA MRT STATION,497.245324
1,1 AKYAB ROAD,ROCHOR MRT STATION,222.262798
2,1 ALEXANDRA VIEW,REDHILL MRT STATION,148.386568
3,1 ALEXANDRA VIEW,CHINATOWN MRT STATION,56.788496
4,1 AMBER GARDENS,LAVENDER MRT STATION,424.647341
...,...,...,...
17265,9L SURIN AVENUE,KOVAN MRT STATION,836.321343
17266,9Q HILLCREST ROAD,TAN KAH KEE MRT STATION,418.045423
17267,9Q HILLCREST ROAD,ROCHOR MRT STATION,231.203830
17268,C-1 HOLLAND PARK,FARRER ROAD MRT STATION,981.703454


In [12]:
# groupby 'InputID' and min 'Distance'

df_MRTdist = df_MRTdist.groupby('InputID', as_index=False).agg({'Distance': 'min'})

In [13]:
#rename "Distance" to "MRT_Distance"
df_MRTdist = df_MRTdist.rename(columns={'Distance': 'MRT_Distance'})
df_MRTdist


,InputID,MRT_Distance
0,1 AKYAB ROAD,222.262798
1,1 ALEXANDRA VIEW,56.788496
2,1 AMBER GARDENS,236.388779
3,1 AMBER ROAD,131.625919
4,1 ANG MO KIO STREET 66,159.594288
...,...,...
8572,9E HILLCREST ROAD,231.203830
8573,9G YUAN CHING ROAD,156.027065
8574,9L SURIN AVENUE,198.981838
8575,9Q HILLCREST ROAD,231.203830


In [14]:
#merge df2 'InputID' with df_MRTdist 'InputID'
df2 = df2.merge(df_MRTdist, left_on='InputID', right_on='InputID', how='left')

In [15]:
#merge df1 'street_address' with df2 'Street_Address'
df1 = df1.merge(df2, left_on='Street_Address', right_on='Street_Address', how='left')

In [16]:
#drop Null values in 'MRT_Distance' and 'signed_distance'
df1 = df1.dropna(subset=['MRT_Distance', 'signed_distance'])


In [17]:
#check headers of df1
df1.columns

Index(['Project Name', 'Transacted Price ($)', 'Area (SQFT)',
       'Unit Price ($ PSF)', 'Sale Date', 'Address', 'Type of Sale',
       'Type of Area', 'Area (SQM)', 'Unit Price ($ PSM)', 'Nett Price($)',
       'Property Type', 'Number of Units', 'Tenure', 'Completion Date',
       'Purchaser Address Indicator', 'Postal Code', 'Postal District',
       'Postal Sector', 'Planning Region', 'Planning Area', 'Street_Address',
       'count', 'InputID', 'TargetID', 'Distance', 'Polygon',
       'signed_distance', 'MRT_Distance'],
      dtype='object')

In [18]:
df1['Planning Area'].unique()

array(['Outram', 'Serangoon', 'Bedok', 'Queenstown', 'Bukit Timah',
       'Hougang', 'Ang Mo Kio', 'Pasir Ris', 'Toa Payoh', 'Punggol',
       'Sengkang', 'Geylang', 'Clementi', 'Downtown Core',
       'Bukit Panjang', 'Bishan', 'Kallang', 'Marine Parade',
       'Singapore River', 'Yishun', 'Choa Chu Kang', 'Bukit Merah',
       'Sembawang', 'Tampines', 'River Valley', 'Tanglin', 'Bukit Batok',
       'Woodlands', 'Jurong West', 'Novena', 'Southern Islands',
       'Orchard', 'Jurong East', 'Mandai', 'Newton', 'Rochor', 'Museum',
       'Changi  ', 'Tengah', 'Sungei Kadut', 'Paya Lebar'], dtype=object)

In [19]:
#check responses in "Property Type" of df1
df1['Property Type'].value_counts()

Property Type
Condominium              20511
Apartment                17107
Executive Condominium     5953
Terrace House             2074
Semi-Detached House        979
Detached House             378
Name: count, dtype: int64

# RDD

In [20]:
df1['Unit Price ($ PSM)'] = df1['Unit Price ($ PSM)'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df1['Area (SQM)'] = df1['Area (SQM)'].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [ ]:
# Prepare data for RDD analysis
print("Data preparation for RDD...")

# Remove rows with missing key variables
rdd_data = df1.dropna(subset=['Unit Price ($ PSM)', 'signed_distance', 'Polygon', 'MRT_Distance', 'Area (SQM)', 'Property Type', 'Completion Date']).copy().reset_index()

# Create treatment variable (1 if inside historic boundary, 0 if outside)
rdd_data['treatment'] = (rdd_data['Polygon'] == 1).astype(int)

# Log transform the price for better model fit
rdd_data['log_price'] = np.log(rdd_data['Unit Price ($ PSM)'])

rdd_data['area'] = rdd_data['Area (SQM)']

#create month variable from 'Sale Date' by selecting the [3:] characters
rdd_data['sale_date'] = df1['Sale Date'].str[3:]
rdd_data['sale_year'] = rdd_data['sale_date'].str[4:]

# Create running variable (distance to boundary)
rdd_data['distance'] = rdd_data['signed_distance']

# create reclaimed dummy if "Street_Address" contains 'MARINA' or 'SHENTON' or 'RHU'
rdd_data['reclaimed'] = rdd_data['Street_Address'].apply(lambda x: 1 if any(keyword in x for keyword in ['MARINA', 'SHENTON', 'RHU']) else 0)

# create landed dummy if 'Property Type' is 'Terrace House' pr 'Semi-Detached House' or 'Detached House'
rdd_data['landed'] = rdd_data['Property Type'].apply(lambda x: 1 if x in ['Terrace House', 'Semi-Detached House', 'Detached House'] else 0)

print(f"Sample size: {len(rdd_data)}")
print(f"Treatment group (inside): {rdd_data['treatment'].sum()}")
print(f"Control group (outside): {len(rdd_data) - rdd_data['treatment'].sum()}")
print(f"Distance range: {rdd_data['distance'].min():.2f} to {rdd_data['distance'].max():.2f}")

# if completion date is "Uncompleted", set to 2024
for i in range(0,len(rdd_data)):
    if rdd_data.at[i,'Completion Date'] == 'Uncompleted':
        rdd_data.at[i,'Completion Date'] = 2024
    elif rdd_data.at[i,'Completion Date'] == '-': # remove this data
        rdd_data = rdd_data.drop(i)
    elif isinstance(rdd_data.at[i,'Completion Date'], str):
        rdd_data.at[i,'Completion Date'] = int(rdd_data.at[i,'Completion Date'])
rdd_data['building_age'] = 2024 - rdd_data['Completion Date']
rdd_data['building_age'] = rdd_data['building_age'].astype(int)

# Basic summary statistics
print("\nSummary Statistics by Treatment Status:")
print(rdd_data.groupby('treatment')[['log_price', 'Unit Price ($ PSM)', 'distance']].describe())

Data preparation for RDD...
Sample size: 47002
Treatment group (inside): 1727
Control group (outside): 45275
Distance range: -15616.93 to 1086.18

Summary Statistics by Treatment Status:
          log_price                                                       \
              count       mean       std       min        25%        50%   
treatment                                                                  
0           44076.0   9.852091  0.308436  7.907284   9.621456   9.853982   
1            1657.0  10.158333  0.257689  9.314520  10.000206  10.161960   

                                Unit Price ($ PSM)                ...  \
                 75%        max              count          mean  ...   
treatment                                                         ...   
0          10.093984  11.092276            44076.0  19903.848149  ...   
1          10.337929  10.969904             1657.0  26653.716355  ...   

                            distance                              

In [22]:
rdd_data['sale_date'].unique()

array(['Jan 2024', 'Feb 2024', nan, 'Mar 2024', 'Apr 2024', 'May 2024',
       'Jun 2024', 'Jul 2024', 'Aug 2024', 'Sep 2024', 'Oct 2024',
       'Nov 2024', 'Dec 2024', 'Jan 2023', 'Feb 2023', 'Mar 2023',
       'Apr 2023', 'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023',
       'Sep 2023', 'Oct 2023', 'Nov 2023', 'Dec 2023'], dtype=object)

In [35]:
rdd_data['sale_year'] = rdd_data['Sale Date'].str[7:]
rdd_data['sale_year'].unique()

array(['2024', '2023'], dtype=object)

In [23]:
# Choose a bandwidth (e.g., 500 meters around the boundary)
bandwidth = 100
rdd_sample = rdd_data[abs(rdd_data['distance']) <= bandwidth]

# Run OLS regression: log_price ~ treatment + distance + distance^2
rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2

model = smf.ols('log_price ~ treatment + distance + reclaimed + building_age + MRT_Distance', data=rdd_sample).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     234.4
Date:                Tue, 08 Jul 2025   Prob (F-statistic):          1.96e-116
Time:                        13:59:51   Log-Likelihood:                 195.86
No. Observations:                 544   AIC:                            -381.7
Df Residuals:                     539   BIC:                            -360.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       10.2898      0.035    292.510   

C:\Users\THW_9\AppData\Local\Temp\ipykernel_19916\2014186640.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2


In [37]:
# Choose a bandwidth (e.g., 500 meters around the boundary)
bandwidth = 100
rdd_sample = rdd_data[abs(rdd_data['distance']) <= bandwidth]

# Run OLS regression: log_price ~ treatment + distance + distance^2
rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2

model = smf.ols('log_price ~ treatment + distance + reclaimed + building_age + MRT_Distance + sale_year', data=rdd_sample).fit()

from IPython.display import display
display(model.summary())


C:\Users\THW_9\AppData\Local\Temp\ipykernel_19916\1914438046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     187.2
Date:                Tue, 08 Jul 2025   Prob (F-statistic):          3.08e-115
Time:                        14:13:10   Log-Likelihood:                 195.89
No. Observations:                 544   AIC:                            -379.8
Df Residuals:                     538   BIC:                            -354.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            10.2910      0.036    289.858      0.000      10.221      10.361
sale_year[T.2024]    -0.0039      0.015     -0.263      0.793      -0.033       0.025
treatment             0.0881      0.050      1.774      0.077      -0.009       0.186
distance             -0.0015      0.000     -4.020      0.000      -0.002      -0.001
reclaimed          9.527e-16   7.67e-18    124.247      0.000    9.38e-16    9.68e-16
building_age         -0.0184      0.001    -27.086      0.000      -0.020      -0.017
MRT_Distance      -1.725e-05      0.000     -0.167      0.868      -0.000       0.000
==============================================================================
Omnibus:                        8.295   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.016   Jarque-Bera (JB):                8.309
Skew:                           0.266   Prob(JB):                       0.0157
Kurtosis:                       3.290   Cond. No.                     7.77e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.11e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [25]:
import seaborn as sns

sns.lmplot(
    data=rdd_sample,
    x='distance',
    y='log_price',
    hue='treatment',
    lowess=True,
    aspect=1.5,
    scatter_kws={'s': 30, 'alpha': 0.5}
)
plt.axvline(0, color='red', linestyle='--')
plt.title('RDD: Log Price vs. Distance to Colonial Boundary')
plt.xlabel('Signed Distance to Boundary')
plt.ylabel('Log(House Price)')


ImportError: cannot import name '_is_pandas_dataframe' from 'matplotlib.cbook' (c:\Users\THW_9\anaconda3\envs\tf\lib\site-packages\matplotlib\cbook\__init__.py)

In [ ]:
# Choose a bandwidth (e.g., 500 meters around the boundary)
bandwidth = 125
rdd_sample = rdd_data[abs(rdd_data['distance']) <= bandwidth]

# Run OLS regression: log_price ~ treatment + distance + distance^2
rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2

model = smf.ols('log_price ~ treatment + distance + treatment * distance + building_age + MRT_Distance', data=rdd_sample).fit()
print(model.summary())

C:\Users\THW_9\AppData\Local\Temp\ipykernel_27608\932135138.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.645
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     95.84
Date:                Sun, 06 Jul 2025   Prob (F-statistic):           2.80e-57
Time:                        16:47:57   Log-Likelihood:                 153.14
No. Observations:                 270   AIC:                            -294.3
Df Residuals:                     264   BIC:                            -272.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             10.2245      0

In [ ]:
# Choose a bandwidth (e.g., 500 meters around the boundary)
bandwidth = 500
rdd_sample = rdd_data[abs(rdd_data['distance']) <= bandwidth]

# Run OLS regression: log_price ~ treatment + distance + distance^2
rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2

model = smf.ols('log_price ~ treatment + distance + building_age + MRT_Distance', data=rdd_sample).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     262.6
Date:                Sun, 06 Jul 2025   Prob (F-statistic):          2.93e-141
Time:                        16:39:39   Log-Likelihood:                 272.11
No. Observations:                 759   AIC:                            -534.2
Df Residuals:                     754   BIC:                            -511.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       10.2157      0.024    427.084   

C:\Users\THW_9\AppData\Local\Temp\ipykernel_27608\1146096412.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdd_sample['distance_sq'] = rdd_sample['distance'] ** 2
